In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import logging
import lxml

Now that necessary packages are installed, going to parse the needed url. This URL has stats for each player in the league for the 2030-31 season, as well as contract information only for the year of 2030-31. So I am going to scrape the cap friendly website (Players Browse tab) and get this into a dataframe that I can download and put into the MySQL database

In [2]:
url_v2 = "https://www.capfriendly.com/browse/active/2031?stats-season=2017&display=signing-team,birthday,country,weight,height,weightkg,heightcm,draft,slide-candidate,waivers-exempt,signing-status,expiry-year,performance-bonus,signing-bonus,caphit-percent,aav,length,minors-salary,base-salary,arbitration-eligible,type,signing-age,signing-date,arbitration,extension&hide=skater-stats,goalie-stats&limits=gp-5-90"

req = requests.get(url_v2)
soup = BeautifulSoup(req.content)  # make a soup of html & css from the web page

In [3]:
df = pd.read_html(url_v2, header=0, index_col = 0, na_values=["-"])[0]

In [4]:
df.shape

(22, 32)

After running df.shape, we can see our dataframe has 14 rows (players) and 32 columns (attributes about that player for the 2030-31 season). There are more than 50 players but they are on different url links technically as the table on that specific url only shows 50 players. So we need to retrieve the rest of the players. 

Scraping multiple pages of the main table

In [5]:
info_about_lists = soup.find_all("a", {"class": "whi pagin_r"})  # via devtools we find the element that allows to switch between pages of data
     

In [6]:
print(info_about_lists)  # all links to other pages of data

[]


Now we can use a for loop to parse all the data we have on multiple pages

In [7]:
req = requests.get(url_v2)
soup = BeautifulSoup(req.content)  # make a soup of html & css from the web page


url_start = "https://www.capfriendly.com/browse/active/2031?stats-season=2017&display=signing-team,birthday,country,weight,height,weightkg,heightcm,draft,slide-candidate,waivers-exempt,signing-status,expiry-year,performance-bonus,signing-bonus,caphit-percent,aav,length,minors-salary,base-salary,arbitration-eligible,type,signing-age,signing-date,arbitration,extension&hide=skater-stats,goalie-stats&limits=gp-5-90"


print(f"Start scapring page 1")

time.sleep(1)  # let the page download the results
url = url_start   # we parse the needed page by adding a parameter for url
df = pd.read_html(url, header=0, index_col = 0, na_values=["-"])[0]
df = df.reset_index()  # to have player name as a separate column
print(df.shape[0], f"rows were retrieved from page number 1")

result_df = df

Start scapring page 1
22 rows were retrieved from page number 1


In [8]:
result_df.head(5)

,PLAYER,TEAM,AGE,DATE OF BIRTH,COUNTRY,WEIGHT,HEIGHT,POS,HANDED,DRAFTED,...,EXPIRY,EXP. YEAR,CAP HIT,CAP HIT %,AAV,SALARY,BASE SALARY,MINORS,S.BONUS,P.BONUS
0,1. Nathan MacKinnon,COL,34,"Sep. 1, 1995",Canada,200 lbs - 91 kg,"6'0"" - 183 cm",C,Right,1 - Round 1 - 2013 (COL),...,UFA,2031,"$12,600,000",15.3%,"$12,600,000","$9,900,000","$990,000","$9,900,000","$8,910,000",$0
1,2. William Nylander,TOR,34,"May 1, 1996",Canada,202 lbs - 92 kg,"6'0"" - 183 cm",RW,Right,8 - Round 1 - 2014 (TOR),...,UFA,2032,"$11,500,000",13.8%,"$11,500,000","$10,000,000","$1,000,000","$10,000,000","$9,000,000",$0
2,3. David Pastrnak,BOS,34,"May 25, 1996",Czech Republic,194 lbs - 88 kg,"6'0"" - 183 cm",RW,Right,25 - Round 1 - 2014 (BOS),...,UFA,2031,"$11,250,000",13.6%,"$11,250,000","$9,000,000","$7,000,000","$9,000,000","$2,000,000",$0
3,4. Jonathan Huberdeau,CGY,37,"Jun. 4, 1993",Canada,202 lbs - 92 kg,"6'1"" - 185 cm","LW, RW",Left,3 - Round 1 - 2011 (FLA),...,UFA,2031,"$10,500,000",12.7%,"$10,500,000","$10,500,000","$5,500,000","$10,500,000","$5,000,000",$0
4,5. Sebastian Aho,CAR,32,"Jul. 26, 1997",Finland,176 lbs - 80 kg,"6'0"" - 183 cm",C,Left,35 - Round 2 - 2015 (CAR),...,UFA,2032,"$9,750,000",11.7%,"$9,750,000","$7,200,000","$5,200,000","$7,200,000","$2,000,000",$0


Now, I have player statistics and cap info for the 2030-31 season. I have every player in the entire league for the 2030-31 season!!! So time to export it to a csv, and then upload it into the MySQL database I created. 

In [9]:
result_df.to_csv('Cap Friendly 2030-31 Player Data2.csv', encoding='utf-8')
result_df.to_csv('Cap Friendly 2030-31 Player Data.csv')

The csv file above is a weird file type **Forgot to add the .csv extension above, issue is fixed now. Just have to replace the weird "âœ” with a Yes as when a check mark is scraped off their website, it can not get represented correctly in excel. So just replace that symbol with the text Yes.

Now I have the 2030-31 CapFriendly Player Data all downloaded, time to download the 10 prior seasons(Include that seasons stats and contract information) and 8 seasons after (will include contract information only). Each Season will be done in a separate file to ensure this notebook does not get negatively impacted and to keep code cleaner